In [ ]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import os
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier, XGBRegressor
import warnings

In [ ]:
df = pd.read_csv("full_info.csv")

df_latest = pd.read_csv("latest_years.csv")

In [ ]:
df_opzones = pd.read_csv("opzone_list.csv")

In [ ]:
df = pd.get_dummies(df, prefix = 'state', columns = ['state'])
df_latest = pd.get_dummies(df_latest, prefix = 'state', columns = ['state'])

In [ ]:
for col in df.columns:
    if 'state' in col:
        df[col] = df[col].apply(lambda x: 1 if x == True else 0)

In [ ]:
df_latest = df_latest[df_latest['year'] == df_latest['year'].max()]

In [ ]:
opzones = list(set(df_opzones['County'].values.tolist()))
opzones_formatted = [x for x in df_latest['place'] if any(y in x for y in opzones)]
df_latest = df_latest[df_latest['place'].isin(opzones_formatted)]

In [ ]:
print('mean current value: ', df_latest['average_home_value'].mean())
print('median current value: ', df_latest['average_home_value'].median())
print('25th percentile current value: ', np.percentile(df_latest['average_home_value'], 25))
print('number 200k or less: ', len(df_latest[df_latest['average_home_value'] <= 200000]))

In [ ]:
df_latest_candidates = df_latest.sort_values(by = 'average_home_value').drop(columns = [x for x in df_latest.columns if x != 'place' and\
                                        x != 'year' and x != 'average_annual_rent' and\
                                        x != 'average_home_value' and x != 'total_population' and \
                                                                                        # '3' not in x and\
                                        'roi' not in x and 'vacancy' not in x])

df_latest_candidates = df_latest_candidates[~df_latest_candidates['place'].str.contains('California')]
df_latest_candidates = df_latest_candidates[~df_latest_candidates['place'].str.contains('New York')]
df_latest_candidates = df_latest_candidates[~df_latest_candidates['place'].str.contains('Puerto Rico')]
df_latest_candidates = df_latest_candidates[~df_latest_candidates['place'].str.contains('Hawaii')]
df_latest_candidates = df_latest_candidates[df_latest_candidates['average_home_value'] <= 250000]
df_latest_candidates = df_latest_candidates[df_latest_candidates['total_population'] >= 100000]
df_latest_candidates = df_latest_candidates[df_latest_candidates['vacancy_growth_last_1_years'] < 0]
df_latest_candidates = df_latest_candidates[df_latest_candidates['vacancy_growth_last_3_years'] < df_latest_candidates['vacancy_growth_last_2_years']]
df_latest_candidates = df_latest_candidates[df_latest_candidates['vacancy_growth_last_2_years'] < df_latest_candidates['vacancy_growth_last_1_years']] 
df_latest_candidates.sort_values(by = 'vacancy_rate')

In [ ]:
plt.hist(df_latest['average_home_value'], bins = 35)
plt.title("Average Home Value Distribution")
plt.xlabel("Value (in millions)")
plt.ylabel("Number of Counties")
plt.savefig("Home_Values_Dist")

In [ ]:
df_latest['vacancy_rate'].mean()

In [ ]:
plt.hist(df_latest['vacancy_rate'], bins = 50)
plt.axvline(df_latest['vacancy_rate'].mean(), color = 'r', linestyle = 'dashed', label = 'Mean Vacancy Rate')
plt.axvline(df_latest_candidates['vacancy_rate'].mean(), color = 'k', linestyle = 'dashed', label = 'Candidate Mean Vacancy Rate')
plt.legend()
plt.title("Vacancy Rate Distribution")
plt.xlabel("Vacancy Rate")
plt.ylabel("Number of Counties")
plt.savefig("Vacancy_Dist")

In [ ]:
print('mean current vacancy rate: ', df_latest['vacancy_rate'].mean())
print('median current vacancy rate: ', df_latest['vacancy_rate'].median())
print('25th percentile current vacancy rate: ', np.percentile(df_latest['vacancy_rate'], 25))


In [ ]:
plt.hist(df_latest[df_latest['vacancy_rate'] < 0.2]['vacancy_rate'], bins = 50)

In [ ]:
# MAKE VALI DATA SECOND TO LAST YEAR; TEST DATA LAST YEAR; TRAIN FIRST 6
last_year = df['year'].max()
train = df[df['year'] < last_year - 4]
vali = df[df['year'] == last_year - 1]
test = df[df['year'] == last_year]

In [ ]:
print('training sample - ', len(train))
print('validation sample - ', len(vali))
print('test sample - ', len(test))

In [ ]:
# SCALE NON-PERCENTAGES
std_scl = StandardScaler()

In [ ]:
#X, Y SPLITS
ex_train = train.drop(columns = ['year', 'place', 'roi'])
ex_train_scaled = std_scl.fit_transform(ex_train)
why_train = train['roi']


ex_vali = vali.drop(columns = ['year', 'place', 'roi'])
ex_vali_scaled = std_scl.fit_transform(ex_vali)
why_vali = vali['roi']

ex_test = test.drop(columns = ['year', 'place', 'roi'])
ex_test_scaled = std_scl.fit_transform(ex_test)
why_test = test['roi']

LET'S SET SOME "DUMB" BASELINES; A VERY SIMPLE MODEL AND STANDARD DEV (THIS ONE SHOULD BE HARDER SINCE WE DON'T KNOW AVERAGE OF POPULATION BEFOREHAND)

In [ ]:
mse = metrics.mean_squared_error
mae = metrics.mean_absolute_error

In [ ]:
print('actuals mean - ', why_vali.mean())
print('actuals standard dev - ', why_vali.std())

In [ ]:
vali['average_home_value'] = vali['average_home_value'].astype('float')
vali['home_val_growth_last_3_years'] = vali['home_val_growth_last_3_years'].astype('float')
vali['average_annual_rent'] = vali['average_annual_rent'].astype('float')
vali['rent_growth_last_1_years'] = vali['rent_growth_last_1_years'].astype('float')
vali['rent_growth_last_2_years'] = vali['rent_growth_last_2_years'].astype('float')
vali['rent_growth_last_3_years'] = vali['rent_growth_last_3_years'].astype('float')


In [ ]:
vali['fake_pred'] = round(((vali['average_home_value'] * (1 + vali['home_val_growth_last_3_years']) - vali['average_home_value']) + \
                     (
                        vali['average_annual_rent'] * (1 + vali['rent_growth_last_1_years']) +\
                        vali['average_annual_rent'] * (1 + vali['rent_growth_last_2_years']) +\
                        vali['average_annual_rent'] * (1 + vali['rent_growth_last_3_years'])
                     ))/vali['average_home_value'], 4)

In [ ]:
print('rmse - ', m.sqrt(mse(why_vali, vali['fake_pred'])))
print('mae - ', mae(why_vali, vali['fake_pred']))
vali = vali.drop(columns = ['fake_pred'])

START W/ OUT OF THE BOX PREDICTIONS

In [ ]:
rf = ens.RandomForestRegressor()
# TRAIN/VALI
rf_fit = rf.fit(ex_train, why_train)
rf_preds = rf_fit.predict(ex_vali)
print('rmse - ', m.sqrt(mse(why_vali, rf_preds)))
print('mae - ', mae(why_vali, rf_preds))

# # PERFORMANCE BEEN BAD SO WANNA SHPEEP TRAINING SCORES TOO - ARE WE OVER OR UNDER FITTING?
rf_train_preds = rf.fit(ex_train, why_train).predict(ex_train)
print('training rmse - ', m.sqrt(mse(why_train, rf_train_preds)))
print('training mae - ', mae(why_train, rf_train_preds))

In [ ]:
feature_names = ex_train.columns
importances = rf_fit.feature_importances_
data = {'feature_names': feature_names, 'feature_importance': importances}
rf_df = pd.DataFrame(data)
rf_df.sort_values(by = ['feature_importance'], ascending = False, inplace = True)
rf_df = rf_df.head(20)

sns.barplot(x = rf_df['feature_importance'], y = rf_df['feature_names'], ci = None)

#ADD CHART LABELS
plt.title('Random Forest Feature Importance')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Names')
# plt.savefig("adaboost_fi_diag_chrom_level", bbox_inches = "tight")

VERSUS HYPERPARAMATER TUNED

In [ ]:
%store -r rf_params

In [ ]:
rf_params

In [ ]:
if sum([1 if 'pca__' in x else 0 for x in rf_params]) > 0:
    steps = [('pca', PCA(n_components = rf_params['pca__n_components'])), 
             ('rf', ens.RandomForestRegressor(n_estimators = rf_params['rf__n_estimators'], 
                                          max_depth = rf_params['rf__max_depth'],
                                          criterion = rf_params['rf__criterion']))]
    model_rf = Pipeline(steps = steps)

    
    
else:
    
    model_rf = ens.RandomForestRegressor(n_estimators = rf_params['n_estimators'], 
                                          max_depth = rf_params['max_depth'],
                                          criterion = rf_params['criterion'])
    
# TRAIN/VALI
rf_fit = model_rf.fit(ex_train, why_train)
rf_preds_tuned = rf_fit.predict(ex_vali)
print('rmse - ', m.sqrt(mse(why_vali, rf_preds_tuned)))
print('mae - ', mae(why_vali, rf_preds_tuned))

# PERFORMANCE BEEN BAD SO WANNA SHPEEP TRAINING SCORES TOO - ARE WE OVER OR UNDER FITTING?
rf_train_preds_tuned = model_rf.fit(ex_train, why_train).predict(ex_train)
print('training rmse - ', m.sqrt(mse(why_train, rf_train_preds_tuned)))
print('training mae - ', mae(why_train, rf_train_preds_tuned))

In [ ]:
%store -r ada_params

In [ ]:
if sum([1 if 'pca__' in x else 0 for x in ada_params]) > 0:
    steps = [('pca', PCA(n_components = ada_params['pca__n_components'])), 
             ('ada', ens.AdaBoostRegressor(n_estimators = ada_params['ada__n_estimators'], 
                                           learning_rate = ada_params['ada__learning_rate']))]
             
    ada = Pipeline(steps = steps)
    
else:
    ada = ens.AdaBoostRegressor(n_estimators = ada_params['n_estimators'], 
                                learning_rate = ada_params['learning_rate'])

    
    
# TRAIN/VALI
ada_fit = ada.fit(ex_train, why_train)
ada_preds = ada_fit.predict(ex_vali)
print('rmse - ', m.sqrt(mse(why_vali, ada_preds)))
print('mae - ', mae(why_vali, ada_preds))
               
# PERFORMANCE BEEN BAD SO WANNA SHPEEP TRAINING SCORES TOO - ARE WE OVER OR UNDER FITTING?
ada_train_preds = ada.fit(ex_train, why_train).predict(ex_train)
print('training rmse - ', m.sqrt(mse(why_train, ada_train_preds)))
print('training mae - ', mae(why_train, ada_train_preds))

In [ ]:
if sum([1 if 'pca__' in x else 0 for x in ada_params]) == 0:
    feature_names = ex_train.columns
    importances = ada_fit.feature_importances_
    data = {'feature_names': feature_names, 'feature_importance': importances}
    ada_df = pd.DataFrame(data)
    ada_df.sort_values(by = ['feature_importance'], ascending = False, inplace = True)
    ada_df = ada_df.head(20)

    sns.barplot(x = ada_df['feature_importance'], y = ada_df['feature_names'], ci = None)

    #ADD CHART LABELS
    plt.title('AdaBoost Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')
    # plt.savefig("adaboost_fi_diag_chrom_level", bbox_inches = "tight")

In [ ]:
%store -r xgboost_params

In [ ]:
xgboost_params

In [ ]:
# XGBOOST IS BEING A CRYBABY ABOUT TYPES IDK WHY, BUT WHATEVER I'LL EXPLICITLY FIX LOL
ex_train['average_annual_rent'] = ex_train['average_annual_rent'].astype(float)
ex_train['average_home_value'] = ex_train['average_home_value'].astype(float)
ex_train['average_income'] = ex_train['average_income'].astype(float)

ex_vali['average_annual_rent'] = ex_vali['average_annual_rent'].astype(float)
ex_vali['average_home_value'] = ex_vali['average_home_value'].astype(float)
ex_vali['average_income'] = ex_vali['average_income'].astype(float)



if sum([1 if 'pca__' in x else 0 for x in xgboost_params]) > 0:
    steps = [('pca', PCA(n_components = xgboost_params['pca__n_components'])), 
             ('xg', XGBRegressor(scale_pos_weight = xgboost_params['xg__scale_pos_weight'],
                                  max_depth = xgboost_params['xg__max_depth'], 
                                  eta = xgboost_params['xg__eta']))]
             
    xg = Pipeline(steps = steps)
    
else:
    xg = XGBRegressor(scale_pos_weight = xgboost_params['scale_pos_weight'],
                  max_depth = xgboost_params['max_depth'], 
                  eta = xgboost_params['eta'])

# TRAIN/VALI
xg_fit = xg.fit(ex_train, why_train)
xg_preds = xg_fit.predict(ex_vali)
print('rmse - ', m.sqrt(mse(why_vali, xg_preds)))
print('mae - ', mae(why_vali, xg_preds))

# PERFORMANCE BEEN BAD SO WANNA SHPEEP TRAINING SCORES TOO - ARE WE OVER OR UNDER FITTING?
xg_train_preds = xg.fit(ex_train, why_train).predict(ex_train)
print('training rmse - ', m.sqrt(mse(why_train, xg_train_preds)))
print('training mae - ', mae(why_train, xg_train_preds))

In [ ]:
if sum([1 if 'pca__' in x else 0 for x in xgboost_params]) == 0:
    feature_names = ex_train.columns
    importances = xg_fit.feature_importances_
    data = {'feature_names': feature_names, 'feature_importance': importances}
    xg_df = pd.DataFrame(data)
    xg_df.sort_values(by = ['feature_importance'], ascending = False, inplace = True)
    xg_df = xg_df.head(20)

    sns.barplot(x = xg_df['feature_importance'], y = xg_df['feature_names'], ci = None)

    #ADD CHART LABELS
    plt.title('XGBoost Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')
    # plt.savefig("adaboost_fi_diag_chrom_level", bbox_inches = "tight")

In [ ]:
# SO MODEL PERFORMANCE NOT GREAT, BUT I WANNA SEE WHAT THE TOP PREDICTED AREAS WERE FOR FUN
# AFTER ALL, IF IT'S TYPICALLY OFF BUT THE VERY HIGH PREDS ARE MORE SOLID, THEN THIS ACTUALLY IS STILL USEFUL
# SINCE OUR GOAL IS TO USE THIS TO IDENTIFY TOP CANDIDATES
# RF WAS MOST PERFORMANT SO GOING TO BASE THIS ON THOSE PREDS

vali['predicted_roi'] = rf_preds

In [ ]:
pred_check = vali.sort_values(by = 'predicted_roi', ascending = False)
pred_check = pred_check.drop(columns = [x for x in df.columns if x != 'place' and\
                                        x != 'year' and x != 'average_annual_rent' and\
                                        x != 'average_home_value' and x != 'total_population' and\
                                        'roi' not in x])


In [ ]:
df_ec = vali.copy()
df_ec['error'] = round(df_ec['predicted_roi'] - df_ec['roi'], 3)

In [ ]:
plt.hist(df_ec['error'], bins = 30)
plt.title("Residual Distribution \nValidation Data")
plt.xlabel("Size of Error")
plt.ylabel("Number of Observations")
plt.savefig("Model_Residuals")

CHECKING IN, NONE OF THE MODELS (OFF THE SHELF OR TUNED) ACTUALLY BEAT THE STANDARD DEVIATION OF THE VALIDATION SET'S ROI. THIS MEANS NONE DOES BETTER THAN PREDICTING THE SET'S AVERAGE (THIS IS HARDER THAN IT SOUNDS AS WE DON'T HAVE THE AVERAGE AHEAD OF TIME). THIS LEADS ME TO CONCLUDE THE MODELS ARE OK IF IMPERFECT. THIS SEEMS TO BE IN LINE WITH THE MODEL ERROR BEING FAIRLY NORMAL BUT NOT QUITE ABOUT ZERO.

NEXT, WE CHECK WITH A MORE "REAL WORLD" TEST. REALISTICALLY WE WOULD NOT BE BUYING AND SHORTING ON EVERY CITY'S REAL ESTATE MARKET, SO THE BETTER APPLICATION IS TO SEE WHAT WOULD HAVE HAPPENED IF WE HAD USED THE MODEL TO IDENTIFY TOP CANDIDATES IN THE LATEST YEAR OF TESTABLE DATA. I CONSIDER ANY CANDIDATE IDENTIFIED THAT WOULD HAVE PERFORMED BETTER THAN AVERAGE ACCEPTABLE (THRESHOLD IN CELL BELOW).

In [ ]:
print('predicted roi mean - ', pred_check['predicted_roi'].mean())
print('roi mean - ', pred_check['roi'].mean())

In [ ]:
opzones = list(set(df_opzones['County'].values.tolist()))
opzones_formatted = [x for x in pred_check['place'] if any(y in x for y in opzones)]
pred_check = pred_check[pred_check['place'].isin(opzones_formatted)]

In [ ]:
pred_check.head(25)

In [ ]:
pred_check.sort_values(by = 'roi', ascending = False).head(25)

In [ ]:
[place for place in pred_check.head(25)['place'].values.tolist() if \
 place in pred_check.sort_values(by = 'roi', ascending = False).head(25)['place'].values.tolist()]

SO, 6 OUT OF 25 OF THE TOP IDENTIFIED CANDIDATES WOULD HAVE PERFORMED WORSE THAN AVERAGE, 6 OF THE TOP 25 PERFORMERS WOULD HAVE HAD BELOW AVERAGE PREDICTIONS, AND 1 COUNTY WOULD HAVE BEEN TOP 25 IN EACH. USING THIS TEST OF THE MODEL IT IS CLEAR THAT IT CAN BE USEFUL (THE MAJORITY OF ITS TOP PREDICTIONS WOULD HAVE BEEN ABOVE AVERAGE, AND THE MAJORITY OF THE TOP PERFORMERS WOULD HAVE BEEN PREDICTED ABOVE AVERAGE) BUT WOULD HAVE TO BE USED WITH GREAT CAUTION IN A REAL-WORLD INVESTMENT SETTING, GIVEN THAT ROUGHLY A QUARTER OF THE TIME ITS RECOMMENDATION WOULD HAVE DONE QUITE POORLY.

In [ ]:
ex_total = pd.concat([pd.concat([ex_train, ex_vali]), ex_test])
why_total = pd.concat([pd.concat([why_train, why_vali]), why_test])

In [ ]:
df_latest = df_latest.dropna()
ex_serving = df_latest.drop(columns = ['year', 'place'])
ex_serving_scaled = std_scl.fit_transform(ex_serving)

In [ ]:
rf_serve = rf.fit(ex_total, why_total)
rf_preds = rf_serve.predict(ex_serving)

In [ ]:
df_latest['pred'] = rf_preds

In [ ]:
# MAKE PLACE EASIER FOR API I.E. NO SPACES; WANNA KEEP IN ORIGINAL DF IN CASE I WANT FURTHER ANALYSIS
df_save = df_latest.copy()
df_save['place'] = df_save['place'].apply(lambda x: re.sub(' ', '.', 
                                                           re.sub('St.', 'Saint', 
                                                                  re.sub(' - ', '.', x))))

In [ ]:
# NO NEED TO KEEP STATE DUMMIES FOR API; STATE ALREADY HUMAN-READABLE WITH PLACE COLUMN
df_save = df_save.drop(columns = [x for x in df_save.columns if 'state_' in x])

In [ ]:
df_save.to_csv("latest_yrs_w_preds_county.csv", index = False)